In [1]:
import fitz  # PyMuPDF
import re
from backend import getMeSomeJuicyAnswers

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text




def preprocess_text(text):
    # Remove unwanted characters and extra spaces
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    return text


# Example usage
def getAnswersFromCV(pdf_path):
    cv_text = extract_text_from_pdf(pdf_path)
    preprocessed_text = preprocess_text(cv_text)
    print(preprocessed_text)
    return getMeSomeJuicyAnswers(preprocessed_text)
pdf_path = 'cv.pdf'
print(getAnswersFromCV(pdf_path))

c:\Users\Work\miniconda3\envs\MD\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Work\miniconda3\envs\MD\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).


Qualifications: ['Phd', 'Computer Science', 'Science', 'Artificial Intelligence']
Locations: ['California']
Job listings in California:


FileNotFoundError: no such file: 'cv.pdf'

ner_tag: 
{'O' : 0, 'B-LANG': 1, 'I-LANG': 2, 'B-SKILL': 3, 'I-SKILL': 4}

In [192]:
from datasets import load_dataset

train_ds = load_dataset('csv', data_files='train.csv', split='train')
test_ds = load_dataset('csv', data_files='test.csv', split='train')

print(train_ds)
print(test_ds)

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 512
})
Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 123
})


In [224]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    print(tokenized_inputs)
    word_ids = tokenized_inputs.word_ids(0)
    print(word_ids)  # Map tokens to their respective word.
    label = examples["ner_tags"]
    previous_word_idx = None
    label_ids = []
    i = 0
    for word_idx in word_ids:  # Set the special tokens to -100.
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:  # Only label the first token of a given word.
            i += 1
            label_ids.append(label[i])
        else:
            label_ids.append(-100)
        previous_word_idx = word_idx
    tokenized_inputs["labels"] = label_ids

    return tokenized_inputs

tokenized_train = train_ds.map(
    tokenize_and_align_labels, 
    batched=True,
    remove_columns=train_ds.column_names
)

train_test_split = tokenized_train.train_test_split(test_size=0.2)

print(train_test_split)


c:\Users\Work\miniconda3\envs\MD\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 409
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 103
    })
})


In [233]:
print(train_test_split['train'])

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 409
})


In [242]:
unique_labels = set()
for example in train_test_split['train']['labels']:
    unique_labels.add(example)
print(f"Unique labels in the training set: {unique_labels}")

# Ensure the number of labels matches the dataset
num_labels = len(unique_labels)
print(f"Number of labels: {num_labels}")

Unique labels in the training set: {0, 1, 3, 4, -100}
Number of labels: 5


In [3]:
# Define training arguments
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=4)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_split['train'],
    eval_dataset=train_test_split['test'],
)

# Train the model
trainer.train()

NameError: name 'BertForTokenClassification' is not defined

In [243]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

label_names = unique_labels
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", id2label=id2label, label2id=label2id)

c:\Users\Work\miniconda3\envs\MD\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [244]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_split['train'],
    eval_dataset=train_test_split['test'],
)

# Train the model
trainer.train()

  0%|          | 0/78 [00:00<?, ?it/s]

ValueError: not enough values to unpack (expected 2, got 1)

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

c:\Users\Work\miniconda3\envs\MD\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Work\.cache\huggingface\hub\models--dslim--bert-base-NER. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassific

[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [6]:
import spacy

data = {
    "furniture": ["This text is about chairs.",
               "Couches, benches and televisions.",
               "I really need to get a new sofa."],
    "kitchen": ["There also exist things like fridges.",
                "I hope to be getting a new stove today.",
                "Do you also have some ovens."]
}

# see github repo for examples on sentence-transformers and Huggingface
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe("classy_classification", 
    config={
        "data": data,
        "model": "spacy"
    }
)

print(nlp("I am looking for kitchen appliances.")._.cats)

ValueError: [E002] Can't find factory for 'classy_classification' for language English (en). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, lemmatizer, trainable_lemmatizer, entity_linker, entity_ruler, tagger, morphologizer, ner, beam_ner, senter, sentencizer, spancat, spancat_singlelabel, span_finder, future_entity_ruler, span_ruler, textcat, textcat_multilabel, en.lemmatizer

In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizer
from datasets import load_dataset
import torch

# Load the pre-trained BERT model for question answering
model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

def batch_tokens(inputs, batch_size=512):
    ret = []
    i = 0
    while i < len(inputs):
        end = min(i + batch_size, len(inputs))
        ret.append(inputs[i:end])
        i = end
    return ret

def answer_question(question, batched_context, top_n=10):
    batches = batched_context.map(lambda x: tokenizer.encode_plus(question, x, return_tensors='pt'))

    answers = []
    for input_ids in batches:
        outputs = model(**input_ids)
        answer_start_scores = outputs.start_logits
        answer_end_scores = outputs.end_logits
    
        top_start_indices = torch.topk(answer_start_scores, top_n).indices[0]
        top_end_indices = torch.topk(answer_end_scores, top_n).indices[0]
    
        print(top_start_indices)
        print(top_end_indices)
    
        for index in range(top_n):
                start_index = top_start_indices[index]
                end_index = top_end_indices[index]
                print(start_index, end_index)
                if end_index >= start_index and end_index < start_index + 2:  # Assuming a maximum answer length of 30 tokens
                    answer = tokenizer.convert_tokens_to_string(
                        tokenizer.convert_ids_to_tokens(input_ids[start_index:end_index + 1])
                    )
                    answers.append(answer)
    
    # Remove duplicates and filter empty answers
    answers = list(dict.fromkeys(answers))
    answers = [answer for answer in answers if answer.strip() != ""]
    return answers

# Example usage
context = """Dr. John Smith is a senior researcher in Artificial Intelligence at MIT. 
He holds a PhD in Computer Science from Stanford University. 
Previously, he worked at Google in California."""
question = "What are its skills?"
print(answer_question(question, context))


c:\Users\Work\miniconda3\envs\MD\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


: 

In [13]:
import fitz  # PyMuPDF
import re
from transformers import BertForQuestionAnswering, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def preprocess_text(text):
    # Remove unwanted characters and extra spaces
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = text.split()
    return text

# Example usage
def getAnswersFromCV(pdf_path):
    cv_text = extract_text_from_pdf(pdf_path)
    preprocessed_text = preprocess_text(cv_text)
    batched_text = batch_tokens(preprocessed_text)
    inputs = tokenizer.encode_plus("What are its skills?", preprocessed_text, return_tensors='pt')
    batches = batch_tokens(inputs["input_ids"].tolist()[0])
    print(batches)

pdf_path = 'cvs/cv.pdf'
getAnswersFromCV(pdf_path)


c:\Users\Work\miniconda3\envs\MD\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (615 > 512). Running this sequence through the model will result in indexing errors


[[101, 2054, 2024, 2049, 4813, 1029, 102, 6795, 100, 10870, 3274, 3992, 3076, 100, 100, 100, 2028, 100, 100, 100, 5978, 100, 11586, 6795, 10870, 100, 100, 4813, 18750, 9262, 1039, 100, 100, 16129, 100, 1039, 100, 29296, 5658, 100, 100, 100, 100, 13825, 100, 5426, 5855, 10355, 8434, 7118, 3455, 4155, 5077, 3128, 2394, 3935, 1039, 2055, 2033, 1045, 2572, 1037, 3811, 12774, 1998, 12479, 3076, 2007, 1037, 2307, 3037, 1999, 1996, 2181, 1997, 4730, 1998, 100, 4454, 1045, 7887, 6148, 2047, 7860, 1998, 7357, 1045, 2572, 7568, 2000, 4553, 2047, 4813, 1998, 6611, 1996, 3924, 1045, 2031, 1045, 2903, 2008, 2026, 6896, 12276, 1998, 19732, 2000, 4553, 3084, 2033, 2019, 7812, 4018, 2005, 2151, 2535, 2495, 2152, 2082, 100, 100, 5828, 100, 1062, 100, 5978, 100, 2118, 1997, 100, 1062, 100, 5978, 2023, 5145, 3039, 2033, 2000, 2022, 1037, 2112, 1997, 2536, 3934, 2008, 1045, 2191, 24608, 2583, 2006, 2026, 100, 16770, 100, 2076, 2023, 3934, 1045, 2209, 2116, 3033, 2107, 2004, 100, 100, 2440, 9991, 1998, 213

In [12]:
print(len([['bernard', 'ambrosio', 'georges', 'computer', 'engineer', 'student', 'faat', 'bernardgeorgesgmailcom', 'faph', 'one', 'famapm', 'arker', 'braga', 'portugal', 'falinke', 'din', 'bernard', 'georges', 'fagithub', 'bernardgeorges', 'skills', 'python', 'java', 'c', 'haskell', 'prolog', 'html', 'css', 'c', 'matlab', 'sql', 'net', 'javascript', 'mongodb', 'nextjs', 'postgres', 'illustrator', 'photoshop', 'interests', 'photography', 'gaming', 'cooking', 'traveling', 'basketball', 'languages', 'portuguese', 'native', 'english', 'advanced', 'c', 'about', 'me', 'i', 'am', 'a', 'highly', 'motivated', 'and', 'ambitious', 'student', 'with', 'a', 'great', 'interest', 'in', 'the', 'area', 'of', 'programming', 'and', 'artiﬁcial', 'intelligence', 'i', 'constantly', 'seek', 'new', 'challenges', 'and', 'adventures', 'i', 'am', 'excited', 'to', 'learn', 'new', 'skills', 'and', 'apply', 'the', 'ones', 'i', 'have', 'i', 'believe', 'that', 'my', 'passion', 'dedication', 'and', 'willingness', 'to', 'learn', 'makes', 'me', 'an', 'ideal', 'candidate', 'for', 'any', 'role', 'education', 'high', 'school', 'escola', 'secundaria', 'carlos', 'amarante', 'z', 'braga', 'portugal', 'bachelors', 'university', 'of', 'minho', 'z', 'braga', 'portugal', 'this', 'institution', 'allowed', 'me', 'to', 'be', 'a', 'part', 'of', 'various', 'projects', 'that', 'i', 'make', 'avail', 'able', 'on', 'my', 'github', 'https', 'githubcombernardgeorges', 'during', 'this', 'projects', 'i', 'played', 'many', 'parts', 'such', 'as', 'backend', 'frontend', 'full', 'stack', 'and', 'even', 'project', 'manager', 'acquiring', 'numerous', 'programming', 'skills', 'along', 'side', 'this', 'hardskills', 'i', 'developed', 'better', 'time', 'management', 'decisionmaking', 'public', 'speaking', 'conﬁ', 'dence', 'friendliness', 'listening', 'problem', 'solving', 'working', 'underpressure', 'along', 'many', 'others', 'softskills', 'masters', 'university', 'of', 'minho', 'z', 'present', 'braga', 'portugal', 'other', 'experiences', 'participant', 'ieee', 'student', 'branch', 'project', 'planning', 'sector', 'z', 'present', 'while', 'in', 'this', 'branch', 'i', 'gained', 'some', 'soft', 'skills', 'such', 'as', 'time', 'management', 'and', 'collaborative', 'skills', 'also', 'through', 'some', 'research', 'projects', 'we', 'created', 'i', 'gained', 'a', 'vast', 'collection', 'of', 'knowledge', 'of', 'various', 'areas', 'joined', 'with', 'improvements', 'in', 'my', 'public', 'speaking', 'participant', 'ieee', 'student', 'branch', 'robotics', 'chapter', 'z', 'present', 'in', 'this', 'chapter', 'we', 'have', 'faced', 'a', 'number', 'of', 'projects', 'in', 'the', 'mechanics', 'area', 'because', 'this', 'was', 'a', 'smaller', 'and', 'self', 'ran', 'chapter', 'i', 'learned', 'along', 'with', 'the', 'coding', 'skills', 'developed', 'more', 'about', 'organizational', 'skills', 'such', 'as', 'planning', 'problemsolving', 'and', 'reliability', 'participant', 'cesium', 'student', 'center', 'z', 'present', 'marketing', 'sector', 'unlike', 'my', 'other', 'experiences', 'this', 'brought', 'me', 'a', 'new', 'sense', 'of', 'creativity', 'and', 'social', 'skills', 'as', 'well', 'as', 'a', 'few', 'experiences', 'in', 'illustrator', 'and', 'photo', 'shoots', 'intern', 'premium', 'minds', 'software', 'house', 'z', 'software', 'developer', 'in', 'this', 'internship', 'i', 'worked', 'under', 'the', 'agile', 'method', 'which', 'aloud', 'me', 'to', 'expe', 'rience', 'the', 'professional', 'environment', 'and', 'work', 'dynamics', 'we', 'were', 'charged', 'of', 'making', 'an', 'internal', 'site', 'in', 'which', 'the', 'backend', 'was', 'in', 'java', 'postgres', 'for', 'the', 'database', 'and', 'the', 'frontend', 'in', 'nextjs', 'here', 'i', 'also', 'improved', 'my', 'ability', 'to', 'learn', 'and', 'be', 'more', 'proactive', 'volunteer', 'coder', 'dojo', 'z', 'mentor', 'in', 'this', 'project', 'i', 'worked', 'as', 'mentor', 'for', 'younger', 'generation', 'teaching', 'them', 'about', 'programming', 'and', 'helping', 'them', 'learn', 'and', 'create', 'various', 'projects', 'this', 'gave', 'me', 'the', 'ability', 'to', 'explain', 'and', 'organize', 'my', 'thoughts', 'in', 'simpler', 'ways', 'and', 'im', 'prove', 'my', 'ability', 'to', 'tackle', 'and', 'resolve', 'problems', 'projects', 'artiﬁcial', 'intelligence', 'z', 'developed', 'in', 'python', 'and', 'available', 'on', 'github', 'https', 'githubcombernardgeorgesai_grupo', 'the', 'program', 'executes', 'four', 'search', 'algorithms', 'namely', 'depth', 'first', 'breath', 'first', 'greedy', 'and', 'a', 'on', 'a', 'given', 'map', 'showing', 'the', 'results', 'in', 'a', 'friendly', 'interface', 'allowing', 'to', 'compare', 'racing', 'game', 'z', 'developed', 'in', 'java', 'and', 'available', 'on', 'github', 'https', 'githubcomfbarrosdss_racingmanager', 'this', 'project', 'involve', 'a', 'extensive', 'faze', 'of', 'requirement'], ['analysis', 'and', 'exempliﬁcation', 'in', 'which', 'we', 'worked', 'with', 'various', 'uml', 'diagrams', 'fairex', 'z', 'developed', 'in', 'css', 'html', 'and', 'sql', 'and', 'available', 'on', 'github', 'https', 'githubcombernardgeorgesfairex', 'this', 'project', 'simulated', 'a', 'soft', 'ware', 'engineering', 'project', 'were', 'we', 'had', 'to', 'build', 'both', 'a', 'data', 'base', 'and', 'its', 'user', 'interface', 'allowing', 'the', 'group', 'to', 'experience', 'both', 'a', 'frontend', 'and', 'backend', 'development', 'events', 'enei', 'visitor', 'z', 'this', 'is', 'an', 'annual', 'computer', 'engineering', 'meeting', 'that', 'took', 'place', 'in', 'aveiro', 'in', 'this', 'event', 'i', 'got', 'to', 'go', 'to', 'various', 'workshops', 'talks', 'activities', 'and', 'much', 'more', 'learning', 'about', 'many', 'diﬀerent', 'areas', 'of', 'computing']]
))

2
